In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

# Gymnasium and ALE
import gymnasium as gym
import ale_py

# Stable Baselines3
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, VecMonitor
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CallbackList, CheckpointCallback, EvalCallback

# Wandb Integration
import wandb
from wandb.integration.sb3 import WandbCallback

# Custom constants
import a2c_pacman_constants as constants

C:\Users\propietari\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def make_env(env_id, render_mode=None):
    def _env():
        env = gym.make(env_id, render_mode=render_mode)
        env = Monitor(env, allow_early_resets=True)  
        return env
    return _env

In [ ]:
run = wandb.init(
    project="A2C Pacman",
    config={
        "env_id": constants.env_id,
        "algorithm": constants.algorithm,
        "learning_rate": constants.learning_rate,
        "gamma": constants.gamma,
        "n_steps": constants.n_steps,
        "vf_coef": constants.vf_coef,
        "ent_coef": constants.ent_coef,   
        "max_grad_norm": constants.max_grad_norm,
        "total_timesteps": constants.total_timesteps, 
        "model_name": constants.model_name,
        "export_path": constants.export_path,
        "videos_path": constants.videos_path,
    },
    sync_tensorboard=True,
    save_code=True,  
)


env_id = "ALE/Pacman-v5"  # Pac-Man environment ID
env = DummyVecEnv([make_env(env_id) for i in range(8)])  # Create 8 parallel environments
env = VecMonitor(env) # Log rollout metrics

# Create and configure the A2C model
model = A2C(
    policy = constants.policy,  # Use a convolutional neural network . Pacman is represented in images (better to use CNN rather than MLP)
    env = env,
    learning_rate = constants.learning_rate,  
    gamma = constants.gamma,  
    n_steps = constants.n_steps,  
    vf_coef = constants.vf_coef,  
    ent_coef = constants.ent_coef,  
    max_grad_norm = constants.max_grad_norm,  
    verbose = 2,  # Enable verbose output                 
    tensorboard_log = f"runs/{run.id}",
)

eval_env = DummyVecEnv([make_env(env_id) for i in range(1)])  

eval_callback = EvalCallback(eval_env, best_model_save_path='./logs/',
                             log_path='./logs/', eval_freq=500,
                             deterministic=False, render=False)

checkpoint_callback = CheckpointCallback(
    save_freq=1000,  
    save_path='./checkpoints/',  
    name_prefix="a2c_pacman", 
)

callback_list = CallbackList([WandbCallback(verbose=2, gradient_save_freq=10), eval_callback, checkpoint_callback])

# Train the model
print("Training...")
model.learn(total_timesteps=constants.total_timesteps, log_interval=100, callback=callback_list)

# Save the trained model
model_path = os.path.join(constants.export_path, constants.model_name)
model.save(model_path)
wandb.save(model_path + ".zip")
wandb.finish()

# Close the training environment
env.close()



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: marionapla. Use `wandb login --relogin` to force relogin


C:\Users\propietari\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\stable_baselines3\common\vec_env\vec_monitor.py:44: UserWarning: The environment is already wrapped with a `Monitor` wrapperbut you are wrapping it with a `VecMonitor` wrapper, the `Monitor` statistics will beoverwritten by the `VecMonitor` ones.
  warnings.warn(


Using cpu device
Wrapping the env in a VecTransposeImage.
Training...
Logging to runs/5duywz9g\A2C_1


C:\Users\propietari\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\stable_baselines3\common\callbacks.py:418: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x00000146570AA3E0> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x0000014657164B20>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


Eval num_timesteps=4000, episode_reward=12.80 +/- 2.71
Episode length: 432.80 +/- 44.91
------------------------------------
| eval/                 |          |
|    mean_ep_length     | 433      |
|    mean_reward        | 12.8     |
| time/                 |          |
|    total_timesteps    | 4000     |
| train/                |          |
|    entropy_loss       | -1       |
|    explained_variance | 0.00585  |
|    learning_rate      | 0.0005   |
|    n_updates          | 15       |
|    policy_loss        | -0.328   |
|    value_loss         | 1.51     |
------------------------------------
New best mean reward!
Eval num_timesteps=8000, episode_reward=18.20 +/- 8.86
Episode length: 516.00 +/- 80.23
------------------------------------
| eval/                 |          |
|    mean_ep_length     | 516      |
|    mean_reward        | 18.2     |
| time/                 |          |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       |

KeyboardInterrupt: 